In [9]:
import gograph
import similarity
import importlib
import re
import time
import annotations

import pandas as pd
import numpy as np

In [10]:
def reload_my_modules():
    """Re-import my modules into the IPython shell"""
    importlib.reload(annotations)
    importlib.reload(gograph)
    importlib.reload(similarity)

In [85]:
%%time
reload_my_modules()
fp = 'data/goa_human.gaf'
anno = annotations.Annotations(fp)

CPU times: user 1.1 s, sys: 88.3 ms, total: 1.19 s
Wall time: 1.18 s


/home/xyz/.local/lib/python3.6/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


In [86]:
# num proteins
print(anno.annotations.groupby('DB_Object_ID').size().count())
# num annos per protein
print(anno.annotations.groupby('DB_Object_ID').size().mean())
#
print(anno.annotations.groupby('DB_Object_ID').size().quantile([0, 0.25, 0.5, 0.75, 1]))
# total annos
print(len(anno.annotations))

19735
25.612515834811248
0.00      1.0
0.25      7.0
0.50     15.0
0.75     30.0
1.00    914.0
dtype: float64
505463


In [14]:
%%time
reload_my_modules()
fp = 'data/go-basic.obo'
#fp = 'data/gene_ontology_edit.obo.2019-07-01'
ontology = gograph.OBOParser(fp).parse_ontology()

CPU times: user 1.75 s, sys: 52.2 ms, total: 1.8 s
Wall time: 1.8 s


In [15]:
t = time.time()
ontology.make_ancestry_matrix()
print(time.time()-t)

7.615031719207764


In [16]:
ontology.ancestry_matrix.matrix.setdiag(1)

In [17]:
anno.annotations = anno.annotations.loc[anno.annotations.Aspect == 'P', :]

In [18]:
len(anno.annotations)

4305

In [19]:
importlib.reload(annotations)
ft = annotations.FrequencyTable(anno, ontology.ancestry_matrix)

/home/xyz/Documents/CodeRepos/ggid/annotations.py:52: RuntimeWarning: divide by zero encountered in log10
  self.information_content = -1*np.log10(self.deep_count.sum(axis=0)/self.deep_count.sum())


In [20]:
ft.ic['GO:0008150']
#'GO:0008152'
#'GO:0009987'

1.4513388333085866

In [21]:
ft.index_dict['GO:0120077']

38807

In [95]:
proteins = list(set(list(anno.annotations.DB_Object_Symbol)))
print(len(proteins))

19691


In [96]:
len(set(anno.annotations.GO_ID))

18407

In [150]:
reload_my_modules()

#P25098 GRK2
#P35626 GRK3

sim = similarity.Similarity(anno, ontology, ft, proteins[:500], namespace='P', verbose=True)

In [151]:
###old time (10)
###11.857779264450073

In [152]:
t0 = time.time()
sim.calculate_similarity()
print(time.time() - t0)
print(sim.counter)

54.88329720497131
74882400


In [155]:
54*((20000/500)**2)/3600

24.0

In [ ]:
sim.get_ic_mica('GO:0006807', 'GO:0006807')

In [55]:
anno.annotations.head()

,DB,DB_Object_ID,DB_Object_Symbol,Qualifier,GO_ID,DB_Reference,Evidence_Code,With_or_From,Aspect,DB_Object_Name,DB_Object_Synonym,DB_Object_Type,Taxon,Date,Assigned_By,Annotation_Extension,Gene_Product_Form_ID
0,RNAcentral,URS000000925B_9913,URS000000925B_9913,NaN,GO:0006412,GO_REF:0000108,IEA,GO:0030533,P,Bos taurus tRNA Threonine with anticodon TGT (...,NaN,tRNA,taxon:9913,20200223,GOC,NaN,NaN
2,RNAcentral,URS0000013967_9913,URS0000013967_9913,NaN,GO:0035195,GO_REF:0000115,IEA,Rfam:RF00658,P,Bos taurus (cattle) microRNA 21 (MIR21),NaN,primary_transcript,taxon:9913,20200222,RNAcentral,NaN,NaN
3,RNAcentral,URS0000017685_9913,URS0000017685_9913,NaN,GO:0006412,GO_REF:0000108,IEA,GO:0030533,P,Bos taurus tRNA,NaN,tRNA,taxon:9913,20200223,GOC,NaN,NaN
7,RNAcentral,URS000001C75E_9913,URS000001C75E_9913,NaN,GO:0006412,GO_REF:0000108,IEA,GO:0030533,P,Bos taurus (cattle) tRNA-Tyr,NaN,tRNA,taxon:9913,20200223,GOC,NaN,NaN
9,RNAcentral,URS0000023412_9913,URS0000023412_9913,NaN,GO:0006412,GO_REF:0000108,IEA,GO:0030533,P,Bos taurus tRNA Threonine with anticodon TGT (...,NaN,tRNA,taxon:9913,20200223,GOC,NaN,NaN


In [59]:
dict(anno.annotations.groupby('DB_Object_ID')['GO_ID'].apply(list))

{'URS000000925B_9913': ['GO:0006412'],
 'URS0000013967_9913': ['GO:0035195'],
 'URS0000017685_9913': ['GO:0006412'],
 'URS000001C75E_9913': ['GO:0006412'],
 'URS0000023412_9913': ['GO:0006412'],
 'URS0000031E12_9913': ['GO:0035195'],
 'URS000003F472_9913': ['GO:0006412'],
 'URS0000041F17_9913': ['GO:0006412'],
 'URS000004450C_9913': ['GO:0006412'],
 'URS000005A328_9913': ['GO:0030308'],
 'URS000005A7A9_9913': ['GO:0006412'],
 'URS000005AEAB_9913': ['GO:0006412'],
 'URS0000062160_9913': ['GO:0006412'],
 'URS0000063442_9913': ['GO:0006412'],
 'URS000006DA05_9913': ['GO:0035195'],
 'URS000007146F_9913': ['GO:0006412'],
 'URS0000078C1F_9913': ['GO:0006412'],
 'URS000007C678_9913': ['GO:0006412'],
 'URS000007E37F_9913': ['GO:0006412'],
 'URS000008C729_9913': ['GO:0006412'],
 'URS000008F0CF_9913': ['GO:0035195'],
 'URS0000092302_9913': ['GO:0006412'],
 'URS0000098C9B_9913': ['GO:0006412'],
 'URS000009BE9A_9913': ['GO:0006412'],
 'URS000009FF11_9913': ['GO:0006396'],
 'URS00000A1A88_9913': ['

In [51]:
#do basic optimization

In [17]:
#GRK2 GRK2 4.880422399667924
#GRK2 GRK3 4.457138969013438
#GRK3 GRK2 4.457138969013437
#GRK3 GRK3 5.106886306085221
#*
#GRK2 GRK2 4.790863646685823
#GRK2 GRK3 4.3652520397371095
#GRK3 GRK2 4.36525203973711
#GRK3 GRK3 5.014008641330033